HEALTH 14 : Disease symptoms prevention

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
df_Data=pd.read_csv('Dataset/dataset.csv')
df_Desc=pd.read_csv('Dataset/symptom_Description.csv')
df_Prec=pd.read_csv('Dataset/symptom_precaution.csv')
df_Sev=pd.read_csv('Dataset/symptom_severity.csv')

'''
df_Data.head()
df_Desc.head()
df_Prec.head()
df_Sev.head()
'''

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5
